## Raytracing Fundamentals

In this tutorial, we will show the fundamentals of using prysm to perform sequential raytracing.  At the moment, this capability is in the experimental submodule, which provides no guarantees of testing, documentation, or future compatability.  However, the core is based on the classic Spencer and Murty paper in JOSA 1961, and is known to work correctly under limited tests.

Raytracing begins by importing a few pieces of machinery,

In [ ]:
import numpy as np

from prysm.experimental.raytracing.surfaces import Surface
from prysm.experimental.raytracing.spencer_and_murty import raytrace
from prysm.experimental.raytracing.raygen import generate_collimated_ray_fan
from prysm.experimental.raytracing.opt import paraxial_image_solve
from prysm.experimental.raytracing.plotting import plot_rays, plot_optics

The first "unit" of a rayrace is the prescription, a series of surfaces with what should be a familiar description of their geometry, and a less familiar global position `P` for the local coordinate origin.

The local coordinate origin is taken to be in z if only a single number is given, This makes the most common case of coaxial designs more ergonomic.

We'll make our prescription a single reflective sphere for now

In [ ]:
mirror_semidiameter = 4
pres = [
    #          curvature reflect|refract position n(wvl), rotation (None=non-tilted)
    Surface.sphere(-0.05, 'reflect', P=5, n=None, R=None)
]

and use a paraxial image solve to add another planar surface at the focus.  This solve needs to known either the entrance pupil diameter, or the object position and object NA, to work.  We're using a collimated input, so we provide an EPD of 2 mm.

In [ ]:
P_img = paraxial_image_solve(pres, z=0, epd=2*mirror_semidiameter)
pres.append(Surface.plane(P_img, 'eval'))

Now we can view a raytrace through the prescription.  The fundamental unit of raytracing is a ray, which is defined as a pair of length 3 vectors,
$$
\begin{align}
P &= \langle X,Y,Z \rangle \\
S &= \langle k,l,m \rangle
\end{align}
$$

$P$ is the location of the ray, and $S$ its direction cosines.  You can simply give the raytrace function any $P$ and $S$, and it will trace that one ray.  Or, you can provide it an ensemble of $N$ $P$ and $S$ with a pair of arrays of shape `(N,3)`, and it will trace all the rays at once.  To create a fan of rays, we can use one of the handy generation functions,

In [ ]:
# starting z is not important, since the ray it collimated.
# it will only affect the raytrace plots
P, S = generate_collimated_ray_fan(nrays=4, maxr=mirror_semidiameter, z=0)
np.set_printoptions(precision=2, suppress=True)
print(P)

Then we simply trace the rays through the prescription, collecting the history of $P$ and $S$ for each ray through each surface,

In [ ]:
# a wavelength is needed, though it does not matter
# since this is all-reflective
phist, shist = raytrace(pres, P, S, 0.6328)
fig, ax = plot_rays(phist)
plot_optics(pres, phist, fig=fig, ax=ax)

We can trace another field point, one offset by a few degrees in the y axis:

In [ ]:
# 10 degrees clockwise advance in y
P2, S2 = generate_collimated_ray_fan(nrays=4, maxr=mirror_semidiameter, z=0, yangle=10)
phist2, shist2 = raytrace(pres, P2, S2, 0.6328)

fig, ax = plot_rays(phist)
plot_rays(phist2, c='b', fig=fig, ax=ax)
plot_optics(pres, phist, fig=fig, ax=ax)

Notice that these rays hit the surface lower on average than the red, on-axis ray bundle.  At the moment, there is nothing in prysm's ray tracing code that knows about aperture stops, and there is no automatic ray-aiming.  "Manual" ray-aiming does exist, and will be covered in a more advanced tutorial.  Because no bounding geometry was specified for the surface, `plot_optics` used the extrema of the raytrace as the surface bound.  Since it was given only the trace history for the off-axis bundle, it does not appear to cover some of the on-axis bundle.

## Wrap-Up

In this tutorial, we showed how to trace rays through a spherical reflecting mirror using prysm.  In it, we showed how to create an optical prescription, use paraxial image solves to locate the image, and plot ray fans from on and off-axis bundles in the same plot.  More advanced tutorials will cover how to construct the prescription for more complex systems and use the raytrace results to perform analysis.